In [2]:
from smpc_secrets import ShamirSecretSharing, AdditiveSecretSharing, Vandermonde, P, RandPoly

In [ ]:
import numpy as np
from numpy.linalg import inv
from scipy.interpolate import lagrange

## Description

In [4]:
# parameters
n=7 # number of parties
t=3 # 2t+1 <= n, subset of the parties that would reconstruct the secret of the multiplication
p=11 # suposed to be large integer value

In [6]:
# Parties
Alice = ShamirSecretSharing(n=n, t=t, p=p)
Bob = ShamirSecretSharing(n=n, t=t, p=p)
Charlie = ShamirSecretSharing(n=n, t=t, p=p)

<bound method ShamirSecretSharing.poly of <secrets.ShamirSecretSharing object at 0x7ff508d9b2e0>>

## Generate Shares

In [56]:
a=Alice.generate(2)
b=Bob.generate(3)
c=Charlie.generate(4)
a,b,c

([(0, 2), (1, 14), (2, 44), (3, 92), (4, 158), (5, 242), (6, 344), (7, 464)],
 [(0, 3), (1, 16), (2, 33), (3, 54), (4, 79), (5, 108), (6, 141), (7, 178)],
 [(0, 4), (1, 10), (2, 24), (3, 46), (4, 76), (5, 114), (6, 160), (7, 214)])

In [58]:
# Polynomials
alice_poly = RandPoly(name="a", n=t,p=p, R=Alice.R)
bob_poly = RandPoly(name="b", n=t,p=p, R=Bob.R)
charlie_poly = RandPoly(name="c", n=t,p=p, R=Charlie.R)

alice_poly, bob_poly, charlie_poly

(a(x)=2+3x+9x^2, b(x)=3+11x+2x^2, c(x)=4+2x+4x^2)

In [59]:
# Shares
alice_shares = (a[1], b[1], c[1]), (a[4], b[4], c[4])
bob_shares = (a[2], b[2], c[2]), (a[5], b[5], c[5])
charlie_shares = (a[3], b[3], c[3]), (a[6], b[6], c[6])

# honest-but-curuous
server_shares = (a[7], b[7], c[7])

alice_shares, bob_shares, charlie_shares, server_shares

((((1, 14), (1, 16), (1, 10)), ((4, 158), (4, 79), (4, 76))),
 (((2, 44), (2, 33), (2, 24)), ((5, 242), (5, 108), (5, 114))),
 (((3, 92), (3, 54), (3, 46)), ((6, 344), (6, 141), (6, 160))),
 ((7, 464), (7, 178), (7, 214)))

In [60]:
# Product of shares
alice_prod = np.array([a[1][1] * b[1][1] * c[1][1], a[4][1] * b[4][1] * c[4][1]])
bob_prod = np.array([a[2][1] * b[2][1] * c[2][1], a[5][1] * b[5][1] * c[5][1]])
charlie_prod = np.array([a[3][1] * b[3][1] * c[3][1], a[6][1] * b[6][1] * c[6][1]])
server_prod = np.array(a[7][1] * b[7][1] * c[7][1])

alice_prod, bob_prod, charlie_prod, server_prod

(array([  2240, 948632]),
 array([  34848, 2979504]),
 array([ 228528, 7760640]),
 array(17674688))

## Re-randomization

In [61]:
alice_rand = RandPoly(name="q1", n=2*t, p=p, fzc=True)
bob_rand = RandPoly(name="q2", n=2*t, p=p, fzc=True)
charlie_rand = RandPoly(name="q3", n=2*t, p=p, fzc=True)

alice_rand, bob_rand, charlie_rand

(q1(x)=9x+7x^2+4x^3+1x^4+5x^5+9x^6,
 q2(x)=3x+7x^2+7x^3+9x^4+1x^5+8x^6,
 q3(x)=7x+7x^2+10x^3+7x^4+9x^5+2x^6)

In [62]:
q1 = alice_rand.poly
q2 = bob_rand.poly
q3 = charlie_rand.poly

In [63]:
alice_q = np.array([q1(1) + q2(1) + q3(1), q1(4) + q2(4) + q3(4)])
bob_q = np.array([q1(2) + q2(2) + q3(2), q1(5) + q2(5) + q3(5)])
charlie_q = np.array([q1(3) + q2(3) + q3(3), q1(6) + q2(6) + q3(6)])
server_q = np.array([q1(7) + q2(7) + q3(7)])

alice_q, bob_q, charlie_q, server_q

(array([  112, 99292]),
 array([  2258, 357620]),
 array([  19686, 1030542]),
 array([2536618]))

In [64]:
alice_s = alice_prod + alice_q
bob_s = bob_prod + bob_q
charlie_s = charlie_prod + charlie_q

server_s = server_prod + server_q

alice_s, bob_s, charlie_s, server_s

(array([   2352, 1047924]),
 array([  37106, 3337124]),
 array([ 248214, 8791182]),
 array([20211306]))

In [65]:
# S = np.concatenate((alice_s, bob_s, charlie_s))
S = np.array([alice_s[0], bob_s[0], charlie_s[0], alice_s[1], bob_s[1], charlie_s[1]])
S = np.append(S, server_s)
S, S.shape

(array([    2352,    37106,   248214,  1047924,  3337124,  8791182,
        20211306]),
 (7,))

## Degree Reduction

In [66]:
# Vandermonde Matrix
B = np.array(Vandermonde(n=n))

# Inverse Vandermonde Matrix
Binv = inv(B)
B, Binv

(array([[     1,      1,      1,      1,      1,      1,      1],
        [     1,      2,      4,      8,     16,     32,     64],
        [     1,      3,      9,     27,     81,    243,    729],
        [     1,      4,     16,     64,    256,   1024,   4096],
        [     1,      5,     25,    125,    625,   3125,  15625],
        [     1,      6,     36,    216,   1296,   7776,  46656],
        [     1,      7,     49,    343,   2401,  16807, 117649]]),
 array([[ 7.00000000e+00, -2.10000000e+01,  3.50000000e+01,
         -3.50000000e+01,  2.10000000e+01, -7.00000000e+00,
          1.00000000e+00],
        [-1.11500000e+01,  4.39500000e+01, -7.90833333e+01,
          8.20000000e+01, -5.02500000e+01,  1.69833333e+01,
         -2.45000000e+00],
        [ 7.08888889e+00, -3.27416667e+01,  6.48333333e+01,
         -7.06944444e+01,  4.46666667e+01, -1.54083333e+01,
          2.25555556e+00],
        [-2.31250000e+00,  1.18333333e+01, -2.53958333e+01,
          2.93333333e+01, -1.927083

## Linear Projection Matrix

In [67]:
Proj = np.array(P(n=n,t=t))
P(n=7,t=3)

[[1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]

## Matrix of product points reduction 

In [68]:
A = B @ Proj @ Binv
A

array([[ 2.93888889e+00, -9.79166667e+00,  2.07500000e+01,
        -2.36944444e+01,  1.54166667e+01, -5.42500000e+00,
         8.05555556e-01],
       [ 1.30555556e+01, -6.40666667e+01,  1.36166667e+02,
        -1.53777778e+02,  9.91666667e+01, -3.46666667e+01,
         5.12222222e+00],
       [ 3.73500000e+01, -1.83825000e+02,  3.81250000e+02,
        -4.25250000e+02,  2.72250000e+02, -9.47250000e+01,
         1.39500000e+01],
       [ 7.58222222e+01, -3.69066667e+02,  7.56000000e+02,
        -8.38111111e+02,  5.34666667e+02, -1.85600000e+02,
         2.72888889e+01],
       [ 1.28472222e+02, -6.19791667e+02,  1.26041667e+03,
        -1.39236111e+03,  8.86416667e+02, -3.07291667e+02,
         4.51388889e+01],
       [ 1.95300000e+02, -9.36000000e+02,  1.89450000e+03,
        -2.08800000e+03,  1.32750000e+03, -4.59800000e+02,
         6.75000000e+01],
       [ 2.76305556e+02, -1.31769167e+03,  2.65825000e+03,
        -2.92502778e+03,  1.85791667e+03, -6.43125000e+02,
         9.4372222

## Randomized polynomial outputs

In [69]:
R = A @ S
R

array([  542.00000006,  1786.00000042,  3756.00000149,  6452.00000286,
        9874.00000548, 14022.00000834, 18896.00001287])

## Lagrange Interpolation

In [70]:
shares = [(i+1, r) for (i, r) in enumerate(R.tolist())]
R[0], shares, shares[0:3]

(542.0000000558794,
 [(1, 542.0000000558794),
  (2, 1786.0000004172325),
  (3, 3756.000001490116),
  (4, 6452.000002861023),
  (5, 9874.000005483627),
  (6, 14022.00000834465),
  (7, 18896.000012874603)],
 [(1, 542.0000000558794), (2, 1786.0000004172325), (3, 3756.000001490116)])

In [71]:
# Lagrange Interpolation for Multiplication 
x = np.arange(1,len(shares[0:3])+1) # prod points indices
y = R[0:3]      # prod points values
f = lagrange(x, y)
x, y, f

(array([1, 2, 3]),
 array([ 542.00000006, 1786.00000042, 3756.00000149]),
 poly1d([363.00000036, 154.99999929,  24.00000041]))

## Degree reduced (and Randomized) polynomial

In [72]:
coef = [(i, r) for (i,r) in enumerate(np.flip(f.coef).tolist())]
p = RandPoly(n=t, R=coef)
p

24.000000406056643+154.9999992940575x+363.0000003557652x^2

## Secret Multiplication Result

In [73]:
np.floor(p.poly(0))

24.0